In [20]:
import pandas as pd
import time
from dataclasses import dataclass
from dataclasses import asdict
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

driver = webdriver.Chrome()

In [2]:
driver.get('https://curso-web-scraping.pages.dev/#/desafio/2')

In [3]:
driver.switch_to.new_window('tab')
driver.switch_to.window(window_name=driver.window_handles[1])
driver.get('https://curso-web-scraping.pages.dev/#/desafio/3')

In [4]:
# Armazenando o nome do usuário
user = driver.find_element(By.CSS_SELECTOR, 'span#usuario').text
user

'Dalton'

In [5]:
driver.window_handles

['E6CE3E890433F56F7600F2E2515B5680', 'C33CD770990ABF6A7F45ADF68CB6A47B']

In [6]:
windows = {
    'desafio_2': 'E6CE3E890433F56F7600F2E2515B5680',
    'desafio_3': 'C33CD770990ABF6A7F45ADF68CB6A47B'
}

In [7]:
driver.switch_to.window(window_name=windows['desafio_2'])

In [11]:
entrada = driver.find_element(By.CSS_SELECTOR, 'main input')
botao = driver.find_element(By.CSS_SELECTOR, 'main button')

entrada.clear()
entrada.send_keys(user)
wait = WebDriverWait(driver=driver, timeout=5, poll_frequency=1)
wait.until(EC.element_to_be_clickable(mark=botao))
botao.click()
#entrada.submit()

In [12]:
usuarios = driver.find_elements(By.CSS_SELECTOR, 'div.users-list > div')
len(usuarios)

10

In [13]:
usuario = usuarios[0]

nome = usuario.find_element(By.TAG_NAME, 'h3')
profissao = usuario.find_element(By.TAG_NAME, 'span')
email = usuario.find_element(By.CSS_SELECTOR, 'ul > li:nth-child(1)')
telefone = usuario.find_element(By.CSS_SELECTOR, 'ul > li:nth-child(2)')
user_name = usuario.find_element(By.CSS_SELECTOR, 'ul > li:nth-child(3)')
estado = usuario.find_element(By.CSS_SELECTOR, 'ul > li:nth-child(4)')

In [14]:
@dataclass
class Usuario:
    nome: str
    profissao: str
    email: str
    telefone: str
    user_name: str
    estado: str

In [15]:
dados_do_usuario = Usuario(
    nome = nome.text,
    profissao = profissao.text,
    email = email.text.split(': ')[1],
    telefone = telefone.text.split(': ')[1],
    user_name = user_name.text.split(': ')[1],
    estado = estado.text.split(': ')[1]
)

In [16]:
dados_do_usuario.nome

'Dalton Pereira'

In [17]:
driver.switch_to.window(window_name=windows['desafio_3'])

In [18]:
input_name = driver.find_element(By.NAME, 'nome')
input_prof = driver.find_element(By.NAME, 'profissao')
input_email = driver.find_element(By.NAME, 'email')
input_tel = driver.find_element(By.NAME, 'telefone')
input_user_name = driver.find_element(By.NAME, 'usuario')
select_state = Select(driver.find_element(By.NAME, 'estado'))

input_name.clear()
input_prof.clear()
input_email.clear()
input_tel.clear()

input_name.send_keys(dados_do_usuario.nome)
input_prof.send_keys(dados_do_usuario.profissao)
input_email.send_keys(dados_do_usuario.email)
input_tel.send_keys(dados_do_usuario.telefone)
input_user_name.send_keys(dados_do_usuario.user_name)
select_state.select_by_visible_text(dados_do_usuario.estado)

In [19]:
#botao_enviar = driver.find_element(By.CSS_SELECTOR, 'button.btn-primary')

#botao_enviar.click()

input_name.submit()

## Automatização do processo

In [ ]:
# Criação da classe Usuario
@dataclass
class Usuario:
    nome: str
    profissao: str
    email: str
    telefone: str
    user_name: str
    estado: str

In [30]:
# Abrindo primeira página e recolhendo menu lateral
driver.switch_to.window(window_name=driver.window_handles[0])
driver.get('https://curso-web-scraping.pages.dev/#/desafio/2')

botao_menu_1 = driver.find_element(By.CSS_SELECTOR, 'button.fixed')
time.sleep(1)
botao_menu_1.click()
time.sleep(1)

# Abrindo segunda página e recolhendo menu lateral
driver.switch_to.new_window('tab')
driver.switch_to.window(window_name=driver.window_handles[1])
driver.get('https://curso-web-scraping.pages.dev/#/desafio/3')

botao_menu_2 = driver.find_element(By.CSS_SELECTOR, 'button.fixed')
time.sleep(1)
botao_menu_2.click()
time.sleep(1)

# Criando dicionario com os ID de cada aba
windows = {
    'pag_busca': driver.window_handles[0],
    'pag_cadastro': driver.window_handles[1]
}

# Loop
while True:
    # Garante que o loop sempre retorne para 2º página
    driver.switch_to.window(window_name=windows['pag_cadastro'])

    try:
        # Registra o nome, muda para a 1º página, insere o nome na barra de busca e clica no botão de buscar
        user = driver.find_element(By.CSS_SELECTOR, 'span#usuario').text
    except:
        # Interrompe o loop não houver mais nomes de usuário
        break

    driver.switch_to.window(window_name=windows['pag_busca'])

    data = []
    
    entrada = driver.find_element(By.CSS_SELECTOR, 'main input')
    entrada.clear()
    entrada.send_keys(user)
    botao_busca = driver.find_element(By.CSS_SELECTOR, 'main button')
    wait = WebDriverWait(driver=driver, timeout=15, poll_frequency=1)
    wait.until(EC.element_to_be_clickable(mark=botao_busca))
    botao_busca.click()

    # Aguarda 5s para renderização das imagens
    driver.implicitly_wait(time_to_wait=5)

    # Registra todos os usuários encontrados na busca
    usuarios = driver.find_elements(By.CSS_SELECTOR, 'div.users-list > div')

    for usuario in usuarios:
        nome = usuario.find_element(By.TAG_NAME, 'h3')
        profissao = usuario.find_element(By.TAG_NAME, 'span')
        email = usuario.find_element(By.CSS_SELECTOR, 'ul > li:nth-child(1)')
        telefone = usuario.find_element(By.CSS_SELECTOR, 'ul > li:nth-child(2)')
        user_name = usuario.find_element(By.CSS_SELECTOR, 'ul > li:nth-child(3)')
        estado = usuario.find_element(By.CSS_SELECTOR, 'ul > li:nth-child(4)')

        dados_do_usuario = Usuario(
            nome = nome.text,
            profissao = profissao.text,
            email = email.text.split(': ')[1],
            telefone = telefone.text.split(': ')[1],
            user_name = user_name.text.split(': ')[1],
            estado = estado.text.split(': ')[1]
        )

        data.append(dados_do_usuario)

    # Alterna para 2° página, preenche os campos com os dados de cada usuário por vez e envia.
    driver.switch_to.window(window_name=windows['pag_cadastro'])

    input_name = driver.find_element(By.NAME, 'nome')
    input_prof = driver.find_element(By.NAME, 'profissao')
    input_email = driver.find_element(By.NAME, 'email')
    input_tel = driver.find_element(By.NAME, 'telefone')
    input_user_name = driver.find_element(By.NAME, 'usuario')
    select_state = Select(driver.find_element(By.NAME, 'estado'))

    for linha in data:
        input_name.clear()
        input_prof.clear()
        input_email.clear()
        input_tel.clear()
        input_user_name.clear()

        input_name.send_keys(linha.nome)
        input_prof.send_keys(linha.profissao)
        input_email.send_keys(linha.email)
        input_tel.send_keys(linha.telefone)
        input_user_name.send_keys(linha.user_name)
        select_state.select_by_visible_text(linha.estado)

        input_name.submit()
